In [1]:
%matplotlib inline
import numpy as np

import jupyter_manim
from manim import *

Manim Community v0.13.1

In [29]:
%%manim -qm -v WARNING phase_est_general

class phase_est_general(Scene):
    config= {
        "x_lim":int,    # define leftmost lowerlimit of x for circuit and gates
        "y_lim":int,    # define up-most limit of y for circuit and gates
        
    }
    def __init__(self, **kwargs):
        super(phase_est_general, self).__init__()       #inheritance from class Scene
        self.x_lim=-6.2
        self.y_lim=-2.5
        self.gap=1
        self.n_qubit=6
        self.sect=6
        self.x_range=[None for i in range(self.sect+1)]
        self.dots=[Dot() for i in range(self.n_qubit)]
        self.circuitline=[[None for i in range(self.n_qubit)] for i in range(self.n_qubit)]   # List to hold all circuit line elements and VGroup  to include all the Mobjects of circuitline
        self.circuit=VGroup()
    def construct(self):
        
        # Algorithm name
        algorithm="Phase Estimation ALgorithm"
        title = Title(algorithm,include_underline=False).set_y(3.1)     #title of the ALgorithm
        self.add_foreground_mobject(title)
        texTemplate = TexTemplate()
        texTemplate.add_to_preamble(r"\usepackage{braket}")

        def create_gate(gate:str,x:int,y:int,control=None,scale=1,textcolor=WHITE,boxcolor=BLUE,fontsize=48,**kwargs):
            if control:
                gate_ = MathTex(gate,color=textcolor,font_size=fontsize).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                if type(control)==list:
                    gate_block=VGroup()
                    gate_block.add(gate_.bg,gate_)
                    for i in range(len(control)):
                        gate_block.add(Dot().set_x(x).set_y(control[i]))
                        gate_block.add(Line([x,control[i],0], [x,y,0]))
                    gate_block.add(gate_.bg,gate_)
                    return gate_block
                else:
                    dot=Dot().set_x(x).set_y(control)
                    control_line= Line([x,control,0], [x,y,0])
                    return VGroup(control_line,gate_.bg,gate_,dot)
            else:
                gate_ = MathTex(gate,color=textcolor,font_size=fontsize).scale(scale+0.5).set_x(x).set_y(y)
                gate_.bg = BackgroundRectangle(gate_,stroke_width=1,color=boxcolor, fill_opacity=1).scale(scale+1)
                return VGroup(gate_.bg,gate_)
        
        def draw_circuitline(sect:int,xrange:list,lines=self.n_qubit,barrier=True,gap=self.gap,y_init=self.y_lim):
            for i in range(lines):
                self.circuitline[sect][i]=Line([xrange[0],y_init+i*gap,-0.5], [xrange[1],y_init+i*gap,-0.5])
                self.circuit.add(self.circuitline[sect][i])
                self.add(self.circuitline[sect][i])
            if barrier:
                barrierline=DashedLine([xrange[1],self.y_lim-0.5,0], [xrange[1],self.y_lim+gap*(self.n_qubit-0.5),0],
                                            dashed_ratio=0.75).set_opacity(0.5)
                self.add(barrierline)
                # self.add(VGroup(circuit,barrierline))
                # return VGroup(circuit,barrier)
        def move_dot(sect: int,dots=self.dots):
            animation=[]
            for j in range(self.n_qubit):
                animation.append(MoveAlongPath(dots[j], self.circuitline[sect][j]))
            # self.play(*animation)
            return animation
        
        def parallelgates(gate,x,sect:int,qubits:list,scale=1):
            hgates=[]
            for i in qubits:
                hgate=create_gate(gate,x=x,y=self.y_lim+i*self.gap,control=None,scale=scale)
                hgates.append(Create(hgate))
            return hgates

        
        # Section I: Uniform Superposition
        depth=sect=1
        self.x_range[0]=[self.x_lim,self.x_lim+0.5+0.75*(depth-1)]
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+1.5+0.5*(depth)]
        
        draw_circuitline(sect,xrange=self.x_range[sect])
        annot=MathTex(r"H^{\otimes t}").scale(1).set_x(self.x_range[sect][0]+1).set_y(self.y_lim-0.3)
        first_reg=MathTex(r"\ket{0}^{\otimes t}",
                tex_template=texTemplate).scale(1).set_x(self.x_range[sect][0]-0.4).set_y(self.y_lim+self.gap*3)
        Psi=MathTex(r"\ket{\psi}",
                tex_template=texTemplate).scale(1).set_x(self.x_range[sect][0]-0.4).set_y(self.y_lim)
        cont_dot=MathTex(r".....").scale(1.2).set_x(self.x_range[sect][0]+0.2).set_y(self.y_lim+self.gap*2.5).rotate(np.pi/2)
        self.add(cont_dot,first_reg,Psi)
        self.play(*move_dot(1),
                    *parallelgates(gate=r"H",sect=1,x=1.01*self.x_range[sect][0]+1,qubits=[_ for _ in range(1,6)],scale=0.5),
                    Create(annot),
                    runtime=2)

        # section II: Controlled phase rotation gates
        sect=2
        depth=3
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+2+1*(depth)]
        draw_circuitline(sect,xrange=self.x_range[sect])
        rot1=create_gate(r"U^{2^{t-1}}",x=self.x_range[sect][0]+0.75,
                            y=self.y_lim,control=self.y_lim+self.gap*5,scale=0.25)
        rot2=create_gate(r"U^{2^{t-2}}",x=self.x_range[sect][0]+2,
                            y=self.y_lim,control=self.y_lim+self.gap*4,scale=0.25)
        rot_d=create_gate(r"...",x=self.x_range[sect][0]+3,y=self.y_lim,
                                textcolor=WHITE,boxcolor=BLACK,scale=0.75)
        cont_dot=MathTex(r".....").scale(1.2).set_x(self.x_range[sect][0]+3.5).set_y(self.y_lim+self.gap*2.5).rotate(np.pi/2)
        self.add(cont_dot)
        rot_t=create_gate(r"U^{2^{0}",x=self.x_range[sect][0]+4,
                            y=self.y_lim,control=self.y_lim+self.gap*1,scale=0.25)
        
        self.play(*move_dot(sect),Create(rot1),Create(rot2),Create(rot_d), Create(rot_t))

        #SECTION III
        sect=3
        depth=0
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+2+1*(depth)]
        draw_circuitline(sect,xrange=self.x_range[sect])  
        box=Rectangle(color=BLUE,height=5.4,width=1,fill_opacity=1).set_x(self.x_range[sect][0]+1).set_y(0)
        text1=MathTex(r"QFT^{\dag}").scale(0.75).set_x(self.x_range[sect][0]+1).set_y(0)
        self.play(*move_dot(sect),Create(box),Create(text1))

        # Section IV
        sect=4
        depth=0
        self.x_range[sect]=[self.x_range[sect-1][1],self.x_range[sect-1][1]+2+1*(depth)]
        draw_circuitline(sect,xrange=self.x_range[sect])
        anim=[]
        for i in range(5):
            meas=create_gate(r"\not \frown",x=self.x_range[sect][0]+1,
                                y=self.y_lim+self.gap*i+1,scale=0.5)
            anim.append(Create(meas))
        self.play(*move_dot(sect), *anim)
        final_state=MathTex(r"\ket{2^t \theta}",
                tex_template=texTemplate).scale(1).set_x(self.x_range[sect][1]+0.6).set_y(self.y_lim+self.gap*3)
        self.play(Create(final_state))
        self.wait(3)


[1, 2, 3, 4, 5, 6]